# Timeseries Singleton Fill¶

Open netcdf file, evaluate time-step and make it consistent

1min, 10min, 1hr, etc

In [1]:
import xarray as xr
import datetime

In [29]:
####### edit these values to point to the file in question
user_data_dir = '/Users/bell/ecoraid/2019/Moorings/19bs8a/final_data_cf/'
filename = '19bs8a_s16_0019m.nc'

# 2*interp_gap
sample_t = {'1T':'2T',
            '30T':'1H',
            '1H':'2H',
           }

interp_gap = sample_t['1H'] #designed sample frequency


### load data and determine timebase

In [30]:
xdf = xr.load_dataset(user_data_dir+filename)
xdf

<xarray.Dataset>
Dimensions:      (time: 18778, depth: 1, latitude: 1, longitude: 1)
Coordinates:
  * time         (time) datetime64[ns] 2019-09-23T23:00:00 ... 2021-11-14T08:...
  * latitude     (latitude) float64 62.2
  * longitude    (longitude) float64 -174.7
  * depth        (depth) float64 19.0
Data variables:
    salinity     (time, depth, latitude, longitude) float64 32.06 ... 32.42
    temperature  (time, depth, latitude, longitude) float64 8.892 ... 1.451
    salinity_QC  (time, depth, latitude, longitude) float64 1.0 1.0 ... 1.0 1.0
Attributes: (12/33)
    Latitude_DegMMddW:                      62 12.00 N
    Longitude_DegMMddN:                     174 40.770 W
    MooringID:                              19BS-8A
    platform_deployment_date:               2019-09-23T22:49:00Z
    platform_deployment_cruise_name:        DY1908
    platform_recovery_date:                 2021-11-14T08:11:00Z
    ...                                     ...
    InstrumentSerialNumber:                 4139
    InstrumentType:                         SBE-16
    date_created:                           2022-12-16T06:22:54Z
    date_modified:                          2023-06-08 14:44:14.229594
    QC_indicator:                           ProbablyGood
    history:                                Trimmed to deployment.\nQCd: 2023...

In [31]:
(xdf.time.diff(dim='time') * 1e-9 ).mean()

<xarray.DataArray 'time' ()>
array(3600, dtype='timedelta64[ns]')

In [27]:
xdf.resample({'time':'1H'}).mean()

<xarray.Dataset>
Dimensions:       (latitude: 1, longitude: 1, depth: 1, time: 18778)
Coordinates:
  * latitude      (latitude) float64 62.2
  * longitude     (longitude) float64 -174.7
  * depth         (depth) float64 30.0
  * time          (time) datetime64[ns] 2019-09-23T23:00:00 ... 2021-11-14T08...
Data variables:
    temperature   (time, depth, latitude, longitude) float64 0.9093 ... 1.468
    conductivity  (time, depth, latitude, longitude) float64 2.77 ... 2.799
    salinity      (time, depth, latitude, longitude) float64 32.24 ... 32.04
    salinity_QC   (time, depth, latitude, longitude) float64 1.0 1.0 ... 1.0 1.0
Attributes: (12/15)
    Latitude_DegMMddW:                      62 12.00 N
    Longitude_DegMMddN:                     174 40.770 W
    MooringID:                              19BS-8A
    platform_deployment_date:               2019-09-23T22:49:00Z
    platform_deployment_cruise_name:        DY1908
    platform_recovery_date:                 2021-11-14T08:11:00Z
    ...                                     ...
    InstrumentSerialNumber:                 2318
    InstrumentType:                         SBE-37
    date_created:                           2022-02-14T10:55:50Z
    date_modified:                          2025-01-16 18:38:13.355587
    QC_indicator:                           ProbablyGood
    history:                                Trimmed to deployment.\nQCd: 2023...

In [28]:
xdf.to_netcdf(user_data_dir+filename,format="NETCDF3_CLASSIC",encoding={'time':{'units':'days since 1900-01-01'}})

/var/folders/6r/n_mpcj7d7pdf9ncqvjy5vb8c0000jm/T/ipykernel_75704/3629195037.py:1: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 1900-01-01'. Resolution of 'hours' needed. Serializing times to floating point instead. Set encoding['dtype'] to integer dtype to serialize to int64. Set encoding['dtype'] to floating point dtype to silence this warning.
  xdf.to_netcdf(user_data_dir+filename,format="NETCDF3_CLASSIC",encoding={'time':{'units':'days since 1900-01-01'}})
